<a href="https://colab.research.google.com/github/YOUR_USERNAME/Digital-Finance-Introduction/blob/main/day_03/notebooks/NB06_Blockchain_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NB06: Blockchain Simulation

**Topic:** 3.2 - Blockchain Mechanics

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Build a Mini-Blockchain**: Construct a working blockchain from scratch with chained blocks
2. **Understand Proof-of-Work**: Implement mining and understand how computational difficulty secures the chain
3. **Detect Tampering**: Understand how blockchain's structure makes it tamper-evident
4. **Appreciate Mining Costs**: Experience firsthand why mining consumes significant computational resources
5. **Validate Chain Integrity**: Implement verification algorithms that check blockchain validity

## Section 1: Setup

We'll build a blockchain using only Python standard library. No external dependencies needed!

In [ ]:
# Import required libraries (all standard library)
import hashlib
import time
import json
from datetime import datetime
from typing import List, Dict, Any

print("✓ Libraries imported successfully!")
print("\nWe're ready to build a blockchain from scratch.")

## Section 2: Building a Block

A blockchain is made up of **blocks**. Each block contains:

- **Index**: Position in the chain
- **Timestamp**: When the block was created
- **Data**: The actual content (transactions, messages, etc.)
- **Previous Hash**: Link to the previous block (creates the "chain")
- **Nonce**: A number we change to find a valid hash (used in mining)
- **Hash**: Digital fingerprint of the entire block

Let's create a Block class:

In [ ]:
class Block:
    """
    A single block in our blockchain.
    """
    
    def __init__(self, index: int, timestamp: float, data: str, previous_hash: str = "0"):
        """
        Initialize a new block.
        
        Args:
            index: Position in the blockchain
            timestamp: When the block was created
            data: Content of the block (transactions, messages, etc.)
            previous_hash: Hash of the previous block (creates the chain)
        """
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = 0  # Used for proof-of-work mining
        self.hash = self.calculate_hash()
    
    def calculate_hash(self) -> str:
        """
        Calculate SHA-256 hash of the block's contents.
        
        Returns:
            Hexadecimal string representation of the hash
        """
        # Combine all block data into a single string
        block_string = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}{self.nonce}"
        
        # Calculate SHA-256 hash
        return hashlib.sha256(block_string.encode()).hexdigest()
    
    def __str__(self) -> str:
        """
        Human-readable representation of the block.
        """
        dt = datetime.fromtimestamp(self.timestamp)
        return f"""\n{'='*70}
Block #{self.index}
{'='*70}
Timestamp:     {dt.strftime('%Y-%m-%d %H:%M:%S')}
Data:          {self.data}
Previous Hash: {self.previous_hash[:16]}...{self.previous_hash[-8:]}
Nonce:         {self.nonce}
Block Hash:    {self.hash[:16]}...{self.hash[-8:]}
{'='*70}
"""

# Test: Create a single block
print("Creating a test block...\n")
test_block = Block(
    index=0,
    timestamp=time.time(),
    data="Hello, Blockchain!",
    previous_hash="0000000000000000"
)

print(test_block)

print("\n💡 Notice how changing ANY part of the block changes its hash!")
print("   This is what makes blockchains tamper-evident.")

### Hash Properties Demonstration

Let's demonstrate the key properties of cryptographic hashes:

In [ ]:
print("\n🔬 HASH PROPERTY EXPERIMENTS\n")
print("="*70)

# Property 1: Deterministic (same input = same output)
print("\n1. DETERMINISTIC - Same input always produces same hash")
block1 = Block(1, time.time(), "Test data", "0")
hash1 = block1.hash
hash2 = block1.calculate_hash()
print(f"   First calculation:  {hash1[:32]}...")
print(f"   Second calculation: {hash2[:32]}...")
print(f"   Match: {hash1 == hash2} ✓")

# Property 2: Avalanche Effect (tiny change = completely different hash)
print("\n2. AVALANCHE EFFECT - Tiny change completely changes hash")
block_a = Block(1, time.time(), "Alice sends 10 BTC to Bob", "0")
block_b = Block(1, time.time(), "Alice sends 11 BTC to Bob", "0")  # Just changed 10 to 11
print(f"   Original:  {block_a.hash[:32]}...")
print(f"   Modified:  {block_b.hash[:32]}...")
print(f"   Only changed one digit, but hash is completely different! ✓")

# Property 3: One-way (can't reverse engineer the input from hash)
print("\n3. ONE-WAY FUNCTION - Can't reverse engineer input from hash")
secret = Block(1, time.time(), "My secret password is: hunter2", "0")
print(f"   Hash: {secret.hash}")
print(f"   Even seeing the hash, you can't determine the original data!")
print(f"   (You'd have to try every possible input - brute force) ✓")

# Property 4: Fixed length output
print("\n4. FIXED LENGTH - Always 64 characters (256 bits) regardless of input size")
short = Block(1, time.time(), "Hi", "0")
long = Block(1, time.time(), "A" * 10000, "0")  # 10,000 characters
print(f"   Short input hash: {short.hash[:32]}... (length: {len(short.hash)})")
print(f"   Long input hash:  {long.hash[:32]}... (length: {len(long.hash)})")
print(f"   Both are exactly 64 characters! ✓")

print("\n" + "="*70)

### The Genesis Block

Every blockchain starts with a **genesis block** - the first block that has no previous block:

In [ ]:
def create_genesis_block() -> Block:
    """
    Create the first block in the chain (Genesis Block).
    
    Returns:
        The genesis block with index 0 and previous_hash "0"
    """
    return Block(
        index=0,
        timestamp=time.time(),
        data="Genesis Block - The beginning of our blockchain",
        previous_hash="0" * 64  # No previous block, so use all zeros
    )

# Create genesis block
genesis = create_genesis_block()
print("\n🌟 GENESIS BLOCK CREATED\n")
print(genesis)

print("\n💡 Fun Fact: Bitcoin's genesis block contains the message:")
print('   "The Times 03/Jan/2009 Chancellor on brink of second bailout for banks"')
print("   This proves the block wasn't created before that date!")

## Section 3: Chaining Blocks

Now let's create a **Blockchain** class that chains blocks together. Each block's hash becomes part of the next block's data, creating an unbreakable chain.

In [ ]:
class Blockchain:
    """
    A simple blockchain implementation.
    """
    
    def __init__(self):
        """
        Initialize blockchain with genesis block.
        """
        self.chain: List[Block] = [create_genesis_block()]
        self.difficulty = 2  # Number of leading zeros required in hash (for mining)
    
    def get_latest_block(self) -> Block:
        """
        Get the most recent block in the chain.
        """
        return self.chain[-1]
    
    def add_block(self, data: str) -> Block:
        """
        Add a new block to the chain.
        
        Args:
            data: Content to store in the block
            
        Returns:
            The newly created block
        """
        previous_block = self.get_latest_block()
        
        new_block = Block(
            index=previous_block.index + 1,
            timestamp=time.time(),
            data=data,
            previous_hash=previous_block.hash  # This creates the "chain"!
        )
        
        self.chain.append(new_block)
        return new_block
    
    def __str__(self) -> str:
        """
        Display the entire blockchain.
        """
        result = "\n" + "#" * 70 + "\n"
        result += "BLOCKCHAIN".center(70) + "\n"
        result += "#" * 70 + "\n"
        
        for block in self.chain:
            result += str(block)
        
        result += "\n" + "#" * 70 + "\n"
        result += f"Total Blocks: {len(self.chain)}\n"
        result += "#" * 70 + "\n"
        
        return result

# Test: Create a blockchain and add some blocks
print("\n🏗️  Creating a blockchain...\n")
my_blockchain = Blockchain()

print("Adding blocks...\n")
my_blockchain.add_block("Alice sends 5 BTC to Bob")
my_blockchain.add_block("Bob sends 2 BTC to Charlie")
my_blockchain.add_block("Charlie sends 1 BTC to Alice")

print(my_blockchain)

print("\n💡 Notice how each block's 'Previous Hash' matches the previous block's 'Block Hash'!")
print("   This creates the chain that makes tampering detectable.")

### Visualizing the Chain

Let's create a simple visualization of how blocks link together:

In [ ]:
def visualize_blockchain(blockchain: Blockchain) -> None:
    """
    Create a visual representation of the blockchain structure.
    """
    print("\n📊 BLOCKCHAIN STRUCTURE VISUALIZATION\n")
    print("="*70)
    
    for i, block in enumerate(blockchain.chain):
        # Block header
        print(f"\n┌─ Block #{block.index} " + "─" * 56 + "┐")
        print(f"│ Data: {block.data[:50]:<50} │")
        print(f"│ Hash: {block.hash[:50]:<50} │")
        print(f"└" + "─" * 68 + "┘")
        
        # Link to next block (if not last)
        if i < len(blockchain.chain) - 1:
            print("  │")
            print("  │ (previous_hash links to block above)")
            print("  ↓")
    
    print("\n" + "="*70)
    print("\n💡 The chain is formed by each block referencing the previous block's hash.")
    print("   Changing any block breaks the chain for all blocks that follow!")

visualize_blockchain(my_blockchain)

## Section 4: Proof of Work

**Proof-of-Work** makes it computationally expensive to create blocks, which prevents spam and makes the chain secure.

The idea:
- A valid block hash must start with a certain number of zeros (the "difficulty")
- We change the `nonce` value repeatedly until we find a hash that meets this requirement
- This is called **mining**

Let's implement mining:

In [ ]:
class MinedBlock(Block):
    """
    A block that requires proof-of-work mining.
    """
    
    def mine_block(self, difficulty: int) -> Dict[str, Any]:
        """
        Mine the block by finding a hash that starts with 'difficulty' zeros.
        
        Args:
            difficulty: Number of leading zeros required
            
        Returns:
            Dictionary with mining statistics
        """
        target = "0" * difficulty  # e.g., "00" for difficulty 2
        
        attempts = 0
        start_time = time.time()
        
        print(f"\n⛏️  Mining block #{self.index}...")
        print(f"   Target: Hash must start with '{target}'")
        print(f"   Searching...")
        
        # Keep trying different nonce values until we find a valid hash
        while not self.hash.startswith(target):
            self.nonce += 1
            self.hash = self.calculate_hash()
            attempts += 1
            
            # Progress indicator
            if attempts % 50000 == 0:
                print(f"   Attempt {attempts:,}... (current hash: {self.hash[:16]}...)")
        
        elapsed_time = time.time() - start_time
        hash_rate = attempts / elapsed_time if elapsed_time > 0 else 0
        
        print(f"\n   ✓ Block mined!")
        print(f"   Winning nonce: {self.nonce:,}")
        print(f"   Final hash: {self.hash}")
        print(f"   Attempts: {attempts:,}")
        print(f"   Time: {elapsed_time:.2f} seconds")
        print(f"   Hash rate: {hash_rate:,.0f} hashes/second")
        
        return {
            'nonce': self.nonce,
            'hash': self.hash,
            'attempts': attempts,
            'time': elapsed_time,
            'hash_rate': hash_rate
        }

# Test mining with different difficulties
print("\n" + "="*70)
print("MINING DEMONSTRATION")
print("="*70)

test_data = "Alice sends 100 BTC to Bob"

# Difficulty 1: Hash must start with "0"
print("\n🔵 DIFFICULTY 1 (1 leading zero)")
easy_block = MinedBlock(1, time.time(), test_data, "0" * 64)
easy_stats = easy_block.mine_block(difficulty=1)

# Difficulty 2: Hash must start with "00"
print("\n🟡 DIFFICULTY 2 (2 leading zeros)")
medium_block = MinedBlock(2, time.time(), test_data, "0" * 64)
medium_stats = medium_block.mine_block(difficulty=2)

# Difficulty 3: Hash must start with "000" (this might take a while!)
print("\n🔴 DIFFICULTY 3 (3 leading zeros) - This may take 10-30 seconds...")
hard_block = MinedBlock(3, time.time(), test_data, "0" * 64)
hard_stats = hard_block.mine_block(difficulty=3)

# Compare difficulties
print("\n" + "="*70)
print("DIFFICULTY COMPARISON")
print("="*70)
print(f"\n{'Difficulty':<15} {'Attempts':<15} {'Time (s)':<15} {'Hash Rate':<20}")
print("-"*70)
print(f"{'1 zero':<15} {easy_stats['attempts']:<15,} {easy_stats['time']:<15.2f} {easy_stats['hash_rate']:>15,.0f} h/s")
print(f"{'2 zeros':<15} {medium_stats['attempts']:<15,} {medium_stats['time']:<15.2f} {medium_stats['hash_rate']:>15,.0f} h/s")
print(f"{'3 zeros':<15} {hard_stats['attempts']:<15,} {hard_stats['time']:<15.2f} {hard_stats['hash_rate']:>15,.0f} h/s")

print("\n💡 Notice:")
print(f"   - Each additional zero increases difficulty by ~16x (16^1 = 16 possibilities per digit)")
print(f"   - Bitcoin currently uses difficulty ~19 leading zeros!")
print(f"   - This makes it EXTREMELY expensive to create fake blocks")

### Why Proof-of-Work Matters

Let's calculate the implications of different difficulty levels:

In [ ]:
print("\n📊 PROOF-OF-WORK ECONOMICS\n")
print("="*70)

# Calculate expected attempts for different difficulties
print("\nExpected attempts to find valid hash:\n")
print(f"{'Difficulty':<15} {'Pattern':<20} {'Expected Attempts':<20} {'Ratio'}")
print("-"*70)

for d in range(1, 8):
    pattern = "0" * d
    expected = 16 ** d  # Each hex digit has 16 possibilities
    ratio = f"16^{d}" if d > 1 else "16"
    print(f"{d:<15} {pattern:<20} {expected:<20,} {ratio}")

print("\n💡 Real-world comparison:")
print(f"   - Your computer: ~{int(easy_stats['hash_rate']):,} hashes/second")
print(f"   - Bitcoin network: ~1,000,000,000,000,000,000 hashes/second (1,000 exahashes or 1 ZH/s)")
print(f"   - Bitcoin difficulty: ~19-20 leading zeros")
print(f"   - Average block time: 10 minutes (by design)")

print("\n⚡ Energy consumption:")
print("   - Bitcoin network uses ~138 TWh/year (according to Cambridge Bitcoin Electricity Consumption Index 2025-2026)")
print("   - This high cost is what secures the network!")
print("   - Attacking Bitcoin would require 51% of this computing power")
print("   - At current prices, that's billions of dollars in hardware + electricity")

print("\n" + "="*70)

## Section 5: Chain Validation

Now let's implement validation to check if a blockchain is valid. A valid blockchain must:

1. Each block's hash must be correctly calculated
2. Each block's `previous_hash` must match the previous block's `hash`
3. Each block's hash must meet the difficulty requirement (if using PoW)

In [ ]:
class ValidatedBlockchain(Blockchain):
    """
    Blockchain with validation and proof-of-work mining.
    """
    
    def add_block(self, data: str) -> Block:
        """
        Add a new mined block to the chain.
        """
        previous_block = self.get_latest_block()
        
        new_block = MinedBlock(
            index=previous_block.index + 1,
            timestamp=time.time(),
            data=data,
            previous_hash=previous_block.hash
        )
        
        # Mine the block before adding it
        new_block.mine_block(self.difficulty)
        
        self.chain.append(new_block)
        return new_block
    
    def is_chain_valid(self) -> Dict[str, Any]:
        """
        Validate the entire blockchain.
        
        Returns:
            Dictionary with validation results
        """
        print("\n🔍 VALIDATING BLOCKCHAIN...\n")
        print("="*70)
        
        issues = []
        
        # Check each block (skip genesis block)
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            
            print(f"\nChecking Block #{i}...")
            
            # Check 1: Is the block's hash correctly calculated?
            calculated_hash = current_block.calculate_hash()
            if current_block.hash != calculated_hash:
                issue = f"Block #{i}: Hash mismatch (block has been tampered with!)"
                print(f"   ✗ {issue}")
                issues.append(issue)
            else:
                print(f"   ✓ Hash correctly calculated")
            
            # Check 2: Does previous_hash link to the previous block?
            if current_block.previous_hash != previous_block.hash:
                issue = f"Block #{i}: Chain broken (previous_hash doesn't match!)"
                print(f"   ✗ {issue}")
                issues.append(issue)
            else:
                print(f"   ✓ Correctly linked to previous block")
            
            # Check 3: Does the hash meet difficulty requirement?
            target = "0" * self.difficulty
            if not current_block.hash.startswith(target):
                issue = f"Block #{i}: Insufficient proof-of-work (hash doesn't meet difficulty)"
                print(f"   ✗ {issue}")
                issues.append(issue)
            else:
                print(f"   ✓ Proof-of-work valid (starts with {self.difficulty} zeros)")
        
        print("\n" + "="*70)
        
        if issues:
            print(f"\n❌ BLOCKCHAIN INVALID - {len(issues)} issue(s) found:\n")
            for issue in issues:
                print(f"   • {issue}")
            return {'valid': False, 'issues': issues}
        else:
            print("\n✅ BLOCKCHAIN VALID - All checks passed!")
            return {'valid': True, 'issues': []}

# Test: Create a valid blockchain
print("\n🏗️  Creating a validated blockchain with mining...\n")
print("(This will take a moment due to mining...)\n")

secure_blockchain = ValidatedBlockchain()
secure_blockchain.difficulty = 2  # 2 leading zeros

secure_blockchain.add_block("Alice sends 10 BTC to Bob")
secure_blockchain.add_block("Bob sends 5 BTC to Charlie")

# Validate the blockchain
validation_result = secure_blockchain.is_chain_valid()

### Demonstrating Tamper Detection

Now let's try to tamper with the blockchain and see what happens:

In [ ]:
print("\n" + "#"*70)
print("TAMPERING DEMONSTRATION")
print("#"*70)

print("\n⚠️  WARNING: We're about to tamper with the blockchain!\n")

# Show original state
print("Original Block #1 data:")
print(f"   '{secure_blockchain.chain[1].data}'\n")

# Tamper with block 1
print("🔧 Tampering: Changing 'Alice sends 10 BTC' to 'Alice sends 100 BTC'...\n")
secure_blockchain.chain[1].data = "Alice sends 100 BTC to Bob"

print("Modified Block #1 data:")
print(f"   '{secure_blockchain.chain[1].data}'\n")

# Validate again
print("\nNow let's validate the blockchain...")
tampered_validation = secure_blockchain.is_chain_valid()

print("\n💡 What happened?")
print("   1. We changed the data in Block #1")
print("   2. But we didn't recalculate its hash")
print("   3. Now the stored hash doesn't match the calculated hash")
print("   4. The validation algorithm detected the tampering!")

print("\n🤔 Could we just recalculate the hash?")
print("   Let's try...\n")

# Recalculate hash
secure_blockchain.chain[1].hash = secure_blockchain.chain[1].calculate_hash()
print("✓ Recalculated hash for Block #1")

# Validate again
print("\nValidating again...")
revalidation = secure_blockchain.is_chain_valid()

print("\n💡 What happened this time?")
print("   1. We recalculated the hash for Block #1")
print("   2. But now Block #2's previous_hash doesn't match!")
print("   3. Changing one block breaks the entire chain after it")
print("   4. To fix the chain, we'd need to recalculate EVERY block after it")
print("   5. With proof-of-work, that's extremely expensive!")

print("\n🔐 This is why blockchain is tamper-evident:")
print("   - Changing old blocks is computationally prohibitive")
print("   - The longer the chain, the more secure early blocks become")
print("   - This is called 'immutability'")

## Section 6: Visualization

Let's create better visualizations of our blockchain:

In [ ]:
def print_chain_summary(blockchain: Blockchain) -> None:
    """
    Print a comprehensive summary of the blockchain.
    """
    print("\n" + "="*70)
    print("BLOCKCHAIN SUMMARY".center(70))
    print("="*70)
    
    print(f"\n📊 Chain Statistics:")
    print(f"   Total Blocks: {len(blockchain.chain)}")
    print(f"   Difficulty: {blockchain.difficulty} (hash must start with {'0' * blockchain.difficulty})")
    
    # Calculate total time span
    if len(blockchain.chain) > 1:
        time_span = blockchain.chain[-1].timestamp - blockchain.chain[0].timestamp
        print(f"   Time Span: {time_span:.2f} seconds")
        print(f"   Avg Block Time: {time_span / (len(blockchain.chain) - 1):.2f} seconds")
    
    print(f"\n📦 Block Details:\n")
    
    for block in blockchain.chain:
        dt = datetime.fromtimestamp(block.timestamp)
        print(f"   Block #{block.index}")
        print(f"   ├─ Time: {dt.strftime('%H:%M:%S')}")
        print(f"   ├─ Data: {block.data[:50]}")
        print(f"   ├─ Nonce: {block.nonce:,}")
        print(f"   └─ Hash: {block.hash[:32]}...")
        print()
    
    print("="*70)

def visualize_chain_integrity(blockchain: Blockchain) -> None:
    """
    Visualize how blocks are linked together.
    """
    print("\n" + "="*70)
    print("CHAIN INTEGRITY VISUALIZATION".center(70))
    print("="*70 + "\n")
    
    for i, block in enumerate(blockchain.chain):
        # Show block
        print(f"┌─ Block #{block.index} " + "─" * 56 + "┐")
        print(f"│ Hash:     {block.hash[:28]}...{block.hash[-16:]} │")
        print(f"│ PrevHash: {block.previous_hash[:28]}...{block.previous_hash[-16:]} │")
        print(f"│ Nonce: {block.nonce:<58} │")
        print(f"└" + "─" * 68 + "┘")
        
        # Check link to next block
        if i < len(blockchain.chain) - 1:
            next_block = blockchain.chain[i + 1]
            if block.hash == next_block.previous_hash:
                print("  ║")
                print("  ╠════ ✓ Link verified")
                print("  ║")
            else:
                print("  ║")
                print("  ╠════ ✗ BROKEN LINK!")
                print("  ║")
    
    print("\n" + "="*70)

# Create a fresh blockchain for visualization
demo_blockchain = ValidatedBlockchain()
demo_blockchain.difficulty = 2

print("\n🎨 Creating blockchain for visualization demo...")
demo_blockchain.add_block("Transaction 1")
demo_blockchain.add_block("Transaction 2")
demo_blockchain.add_block("Transaction 3")

# Show visualizations
print_chain_summary(demo_blockchain)
visualize_chain_integrity(demo_blockchain)

## Section 7: Challenge Exercises

Test your understanding with these challenges!

### Challenge 1: What Happens If You Change a Transaction?

Create a blockchain with 5 blocks, then change the data in block 2. Try to "repair" the blockchain by recalculating hashes. How many blocks do you need to recalculate?

In [ ]:
# YOUR TURN: Complete this challenge!

print("\n🎯 CHALLENGE 1: Tampering and Repair\n")
print("="*70)

# Step 1: Create blockchain with 5 blocks
challenge_chain = ValidatedBlockchain()
challenge_chain.difficulty = 2

print("\nStep 1: Creating blockchain...\n")
for i in range(1, 6):
    challenge_chain.add_block(f"Transaction {i}")

print("\nBlockchain created. Validating...")
challenge_chain.is_chain_valid()

# Step 2: Tamper with block 2
print("\n\nStep 2: Tampering with Block #2...\n")
print(f"Original: {challenge_chain.chain[2].data}")
challenge_chain.chain[2].data = "HACKED TRANSACTION - All coins to hacker!"
print(f"Modified: {challenge_chain.chain[2].data}")

# Step 3: Validate (should fail)
print("\n\nStep 3: Validating tampered blockchain...")
challenge_chain.is_chain_valid()

# Step 4: Try to repair
print("\n\nStep 4: Attempting to repair...\n")

# TODO: Your code here!
# Hints:
# - Recalculate hash for block 2: challenge_chain.chain[2].hash = ...
# - What about block 3? Its previous_hash is now wrong!
# - Do you need to recalculate all blocks after the change?
# - Don't forget to re-mine each block (find valid nonce)!

print("\n💡 Questions to think about:")
print("   1. How many blocks did you need to recalculate?")
print("   2. How long did it take to repair the chain?")
print("   3. What if the chain had 1000 blocks instead of 5?")
print("   4. What if the difficulty was 4 instead of 2?")
print("   5. Why does this make blockchain secure against tampering?")

### Challenge 2: Why Does Difficulty Matter?

Run mining experiments with different difficulties and calculate:
- How much longer does each additional zero take?
- At what difficulty does it become impractical on your computer?

In [ ]:
# YOUR TURN: Complete this challenge!

print("\n🎯 CHALLENGE 2: Difficulty Economics\n")
print("="*70)

difficulties_to_test = [1, 2, 3, 4]  # Add 5 if you're brave!
results = []

for difficulty in difficulties_to_test:
    print(f"\n{'='*70}")
    print(f"Testing difficulty {difficulty} ({difficulty} leading zeros)")
    print(f"{'='*70}")
    
    # TODO: Your code here!
    # Hints:
    # - Create a MinedBlock
    # - Call mine_block(difficulty)
    # - Store the statistics
    # - Compare results
    
    pass  # Replace with your code

# TODO: Create a comparison table of your results
# Show: difficulty, attempts, time, hash rate

print("\n💡 Questions to answer:")
print("   1. What's the ratio between attempts for difficulty N and N+1?")
print("   2. At what difficulty does it take more than 1 minute?")
print("   3. Bitcoin uses ~19 leading zeros. Estimate how long that would take on your computer.")
print("   4. Why don't we just set Bitcoin's difficulty to 1?")
print("   5. How does difficulty adjustment work in Bitcoin?")

### Challenge 3: Calculate Hashes Per Second

Write a function to benchmark your computer's hash rate and compare it to real mining hardware:
- Your computer
- A gaming GPU
- An ASIC miner
- The entire Bitcoin network

In [ ]:
# YOUR TURN: Complete this challenge!

print("\n🎯 CHALLENGE 3: Hash Rate Benchmarking\n")
print("="*70)

def benchmark_hash_rate(duration_seconds: float = 5.0) -> float:
    """
    Benchmark how many hashes per second your computer can calculate.

    Args:
        duration_seconds: How long to run the benchmark

    Returns:
        Hashes per second
    """
    import hashlib
    import time

    # Create test data
    test_data = "benchmark_test_data"
    nonce = 0
    hash_count = 0

    start_time = time.time()
    end_time = start_time + duration_seconds

    # Keep hashing until time runs out
    while time.time() < end_time:
        # Calculate hash with incrementing nonce
        data = f"{test_data}{nonce}".encode()
        hashlib.sha256(data).hexdigest()
        nonce += 1
        hash_count += 1

    elapsed = time.time() - start_time
    hash_rate = hash_count / elapsed if elapsed > 0 else 0

    return hash_rate

# Run benchmark
print("\nBenchmarking your computer... (this will take a few seconds)\n")
your_hash_rate = benchmark_hash_rate(5.0)
print(f"Your computer: {your_hash_rate:,.0f} hashes/second\n")

# Compare to real hardware
print("\n📊 HASH RATE COMPARISON\n")
print("="*70)
print(f"\n{'Hardware':<30} {'Hash Rate':<25} {'Ratio to You'}")
print("-"*70)

comparisons = [
    ("Your Computer", your_hash_rate, 1),
    ("Gaming GPU (RTX 4090)", 1_000_000, None),  # ~1 MH/s
    ("ASIC Miner (Antminer S19)", 110_000_000_000_000, None),  # 110 TH/s
    ("Entire Bitcoin Network", 1_000_000_000_000_000_000_000, None),  # 1000 EH/s (1 ZH/s)
]

for name, hash_rate, ratio in comparisons:
    if ratio is None:
        ratio = hash_rate / your_hash_rate

    # Format hash rate with units
    if hash_rate >= 1e18:
        hr_display = f"{hash_rate/1e18:.1f} EH/s"
    elif hash_rate >= 1e15:
        hr_display = f"{hash_rate/1e15:.1f} PH/s"
    elif hash_rate >= 1e12:
        hr_display = f"{hash_rate/1e12:.1f} TH/s"
    elif hash_rate >= 1e9:
        hr_display = f"{hash_rate/1e9:.1f} GH/s"
    elif hash_rate >= 1e6:
        hr_display = f"{hash_rate/1e6:.1f} MH/s"
    else:
        hr_display = f"{hash_rate:,.0f} H/s"

    print(f"{name:<30} {hr_display:<25} {ratio:,.0f}x")

print("\n💡 Questions to think about:")
print("   1. How long would it take you to mine one Bitcoin block at difficulty 19?")
print("   2. Why do miners use specialized ASIC hardware instead of GPUs?")
print("   3. What percentage of the network's hash rate would you need to attack Bitcoin?")
print("   4. At $0.10/kWh electricity, is mining profitable on your computer?")
print("   5. How does hash rate relate to blockchain security?")

## Summary

In this notebook, you built a working blockchain from scratch and learned:

### Key Concepts:

1. **Blockchain Structure**:
   - Blocks contain data, timestamps, hashes, and links to previous blocks
   - The "chain" is formed by each block referencing the previous block's hash
   - Genesis block is the first block (no previous block)

2. **Cryptographic Hashing**:
   - SHA-256 creates a unique fingerprint for any data
   - Any change to input data completely changes the hash (avalanche effect)
   - Hashes are one-way (can't reverse engineer original data)
   - Fixed length output (64 hex characters = 256 bits)

3. **Proof-of-Work Mining**:
   - Valid blocks must have hashes meeting difficulty requirements
   - Finding valid hash requires trying many nonce values (computational work)
   - Each additional leading zero increases difficulty ~16x
   - This makes creating fake blocks extremely expensive

4. **Tamper Detection**:
   - Changing any block breaks the chain for all subsequent blocks
   - Validation algorithms can detect tampering
   - Repairing the chain requires recalculating all blocks after the change
   - With proof-of-work, this is computationally prohibitive

5. **Security Through Cost**:
   - Mining difficulty adjusts to maintain consistent block time
   - Network security comes from computational cost of creating blocks
   - Attacking blockchain requires 51% of network's computing power
   - This makes Bitcoin one of the most secure networks in existence

### Real-World Implications:

- **Bitcoin**: Uses this exact structure with ~19 leading zeros difficulty
- **Energy Usage**: Proof-of-work consumes enormous energy (by design)
- **Alternatives**: Proof-of-stake (Ethereum 2.0) uses economic stake instead of computation
- **Immutability**: Older blocks become increasingly secure over time
- **Trustlessness**: No central authority needed - math and incentives ensure security

### Next Steps:

- Study real blockchain implementations (Bitcoin, Ethereum)
- Learn about consensus mechanisms (PoW, PoS, DPoS)
- Explore smart contracts and programmable blockchains
- Understand blockchain scalability challenges (trilemma)
- Research Layer 2 solutions and sidechains

### Further Reading:

- [Bitcoin Whitepaper](https://bitcoin.org/bitcoin.pdf) - Satoshi Nakamoto's original paper
- [Blockchain Demo](https://andersbrownworth.com/blockchain/) - Interactive visualization
- [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) - Technical deep dive
- [Bitcoin Energy Consumption](https://digiconomist.net/bitcoin-energy-consumption) - Real-time stats
- [Proof-of-Work vs Proof-of-Stake](https://ethereum.org/en/developers/docs/consensus-mechanisms/) - Ethereum docs